## Training a whole network
So far, we have been training a single layer and we have been following the progress with the help of the Eager mode. If we want to train a whole network, it is much better to leave the Eager mode and to compile the graph. For this, we need to kill the notebook kernel ("Kernel" -> "Restart"). After that, we continue below in the classical TensorFlow mode.  

In [ ]:
import tensorflow as tf
print('TensorFlow version '+tf.__version__)
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

**Task 12:** We will train on CIFAR10, which is contained in `keras.datasets`. Download the dataset, show a sample for each class, convert the labels to one-hot coding, and normalize the images.

In [ ]:
# Load data

from keras.datasets import cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()       

In [ ]:
# Sample from each class

classes = range(10)
i = 0
while classes:
    if Y_train[i][0] in classes:
        visualize(X_train[i])
        classes = [x for x in classes if x!=Y_train[i][0]]
    i += 1

In [ ]:
# Normalize images

objects_X = [X_train, X_test]
normalized_objects_X = [[], []]
for j in range(len(objects_X)):
    obj = objects_X[j]
    for i in range(len(obj)):
            summed_img = np.sum(obj[i], axis = 2)
            norm_image = np.expand_dims(summed_img / (255*3), 2)
            #norm_image = np.expand_dims(summed_img / (255*3), 0)
            normalized_objects_X[j].append(norm_image)
normalized_X_train = np.array(normalized_objects_X[0])
normalized_X_test = np.array(normalized_objects_X[1])

In [ ]:
normalized_X_train[0].shape

In [ ]:
# Hot encoding

hot_Y_train = tf.one_hot(Y_train, len(set(tuple(x) for x in Y_train)))
hot_Y_test = tf.one_hot(Y_test, len(set(tuple(x) for x in Y_test)))


from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder_train = LabelEncoder()
encoder_train.fit(Y_train)
encoded_Y_train = encoder_train.transform(Y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_Y_train = np_utils.to_categorical(encoded_Y_train)

# encode class values as integers
encoder_test = LabelEncoder()
encoder_test.fit(Y_test)
encoded_Y_test = encoder_test.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_Y_test = np_utils.to_categorical(encoded_Y_test)

**Task 13:** Build the depicted LeNet5-inspired model using Keras standard components.
![title](model.png)

We will now train the network. For displaying the performance we define a helper function:

In [ ]:
def plot_model_history(model):
    line_w = 3
    fig_size = (8, 6)
    plt.figure(0)
    plt.plot(model.history['acc'],'r', lw=line_w)
    plt.plot(model.history['val_acc'],'b', lw=line_w)
    plt.rcParams['figure.figsize'] = fig_size
    plt.xlabel("Epoch number")
    plt.ylabel("Accuracy")
    plt.title("Training Accuracy vs Validation Accuracy")
    plt.legend(['Training','Validation'])
    plt.grid(True)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import Adam, rmsprop
from keras.layers import Activation
from keras import regularizers

def base_model(x_train, num_classes):
    model = Sequential()
    print(x_train.shape[1:])
    print(x_train.shape)
    model.add(Conv2D(32, (3,3), padding='same', input_shape=x_train.shape[1:]))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3,3), padding='same'))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    ###################################################################################
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3,3), padding='same'))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    ###################################################################################
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(Activation('elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    ##################################################################################
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    opt = Adam(lr=0.001, decay=1e-6)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model

**Task 14:** Train the defined model for 50 epochs and a suitable batch size and display the result.

In [ ]:
out_trained_LeNet5 = base_model(normalized_X_train, 10).fit(
    normalized_X_train, dummy_Y_train, batch_size = 250, epochs = 150, 
    validation_data = (normalized_X_test, dummy_Y_test))

In [ ]:
plot_model_history(out_trained_LeNet5)

The results can be improved in several ways, e.g., by successively reducing the learning rate (divided by two after 30 and 40 epochs), by data augmentation (`ImageDataGenerator`), and by weight regularization (see above).  
**Extra task:** Try to improve the result using one or several of the mentioned approaches.  
Hint: you need to define a suitable callback function to change the learning rate during training.